### Problem data

## Simple PV injection (diagram $p-q$)

We consider the following simple 2-bus example network as a minimal example, including PV power:

- The 1km cable with the following characteristics: $(\mathrm{R} + \text{ j}\mathrm{X})' = 0.193 + \text{ j} 0.068 \mathrm{~\Omega}/\text{km} $

- Photovoltaic power: $34.2 \text{ kW}$.

- Voltage: $230 \text{ V} $

### Creating the load flow example in pandapower

A three phase or unbalanced load flow is performed by using the function `pp.runpp_3ph(net)`.

In [33]:
import pandapower as pp

# Create empty net
net = pp.create_empty_network()

# Create buses
bus_eg = pp.create_bus(net, vn_kv=0.230, name="Ext. Grid")
bus_pvload = pp.create_bus(net, vn_kv=0.230, name="PV Load")

# Create bus elements
pp.create_ext_grid(net, bus=bus_eg, vm_pu=1.0, name="Grid Connection")
pp.create_load(net, bus=bus_pvload, p_mw=-0.0342/3, name="Photovoltaic power")

# Create branch elements
line_data = {"c_nf_per_km": 0, "r_ohm_per_km": 0.193, "x_ohm_per_km": 0.068, "max_i_ka": 0.400}
pp.create_std_type(net, line_data, "IESE zxYY SE", element='line')

#impedance = pp.create_impedance(net, from_bus=bus1, to_bus=bus2, sn_mva = 0.0100, rft_pu=0.943, xft_pu=0.332)
line = pp.create_line(net, from_bus=bus_eg, to_bus=bus_pvload, length_km=1.0, std_type="IESE zxYY SE", name="Line")

### Data structure

Each dataframe in a `pandapower` net object contains the information about one `pandapower` element, such as line, load transformer etc.

In [34]:
net.bus

,name,vn_kv,type,zone,in_service
0,Ext. Grid,0.23,b,None,True
1,PV Load,0.23,b,None,True


**Note**: We have created a non standard line with the km impedance given by Marc:

In [35]:
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service
0,Line,IESE zxYY SE,0,1,1.0,0.193,0.068,0.0,0.0,0.4,1.0,1,NaN,True


### Power flow simulation

We now run a balanced 3-phase AC Power Flow:

In [42]:
# 1-phase
pp.runpp(net)

# 3-phase
#pp.runpp_3ph(net)

And check out at the results for buses and line:

In [37]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.00000,0.010962,-0.000154
1,1.039897,0.80743,-0.011400,0.000000


#### Switches

We now create an open switch at the load bus:

In [38]:
pp.create_switch(net, bus=bus2, element=line, et="l", closed=True)

0

We can verify if the switch cuts the load bus from power supply, by running a power flow and inspecting the results:

In [39]:
pp.runpp(net, numba=True)
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.00000,0.010962,-0.000154
1,1.039897,0.80743,-0.011400,0.000000


We can see that the voltage at bus 2 is given as NaN. The load does not feed in:

In [40]:
net.res_load

,p_mw,q_mvar
0,-0.0114,0.0


And the line is in open loop operation:

In [41]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,-0.010962,0.000154,0.0114,1.173720e-13,0.000438,0.000154,0.027519,0.027519,0.027519,1.0,0.0,1.039897,0.80743,6.879648
